### 1. References:

Darmet, L., Ladouce, S., & Dehais, F. (2023). **Shortened calibration of code-VEP based BCI by improved deep learning 
architecture and golden subjects pre-training.** *In 11th International IEEE EMBS Conference on Neural Engineering.* 

Source: https://github.com/neuroergoISAE/offline_cVEP_bitwise

Thielen, J., Marsman, P., Farquhar, J., & Desain, P. (2021). **From full calibration to zero training for a code-modulated
visual evoked potentials for brain–computer interface**. *Journal of Neural Engineering, 18(5), 056007.*

Source: https://gitlab.socsci.ru.nl/jthielen/pynt/-/blob/main/examples

Sebastian Nagel

Source: https://github.com/thecodingdad/eeg2code

Teulings, D.E.J. David (2022).  **an-application-of-recurrency-to-eeg2code-deep-learning**

Source: https://gitlab.socsci.ru.nl/D.Teulings/an-application-of-recurrency-to-eeg2code-deep-learning/-/tree/master

In [1]:
# the path to the dataset
path = 'D:/Dataset'

### 2. Imports

In [2]:
 # Imports
import os
import glob
import h5py
import numpy as np
import scipy.io as sio
import mne

### 3. Preprocessing
#### settings

In [3]:
# Subjects to read and preprocess
# subjects = ["sub-01"]  # subject of participants
subjects = [f"sub-{1+i:02d}" for i in range(30)]  # all participants

# Configuration for the preprocessing
fs = 300  # target sampling frequency in Hz for resampling
band_pass = [2.0, 30.0]  # bandpass cutoffs in Hz for spectral filtering
tmin = 0.0  # trial onset in seconds for slicing
tmax = 31.5  # trial end in seconds for slicing

# The screen refresh rate
FR = 60

# Load codes
V = sio.loadmat(os.path.join(path, "resources", "mgold_61_6521_flip_balanced_20.mat"))["codes"]

# Upsample codes from screen framerate to EEG sampling rate
V = np.repeat(V, fs // FR, axis=0).astype("uint8")

#### Preprocess and save data

In [4]:
# The experimental blocks
BLOCKS = ["block_1", "block_2", "block_3", "block_4", "block_5"]

# Loop over subjects
for subject in subjects:

    epochs = []
    labels = []

    # Loop over blocks
    for block in BLOCKS:

        # Find gdf file
        folder = os.path.join(path, "sourcedata", "offline", subject, block, f"{subject}_*_{block}_main_eeg.gdf")
        listing = glob.glob(folder)
        assert len(listing) == 1, f"Found none or multiple files for {subject}_{block}, should be a single file!"
        fname = listing[0]

        # Read raw file
        raw = mne.io.read_raw_gdf(fname, stim_channel="status", preload=True)

        # Spectral band-pass filter
        raw.filter(l_freq=band_pass[0], h_freq=band_pass[1])

        # Downsample
        raw.resample(sfreq=fs)

        # Read events
        events = mne.find_events(raw)

        # Select only the start of a trial
        # N.B. Every 2.1 seconds an event was generated (15 times per trial, plus one 16th "leaking epoch")
        # N.B. This "leaking epoch" is not always present, so taking epoch[::16, :] won't work, unfortunately
        cond = np.logical_or(np.diff(events[:, 0]) < 1.8 * raw.info['sfreq'], 
                             np.diff(events[:, 0]) > 2.4 * raw.info['sfreq'])
        idx = np.concatenate(([0], 1 + np.where(cond)[0]))
        onsets = events[idx, :]

        # Visualize events
        # import matplotlib.pyplot as plt
        # fig, ax = plt.subplots(1, 1, figsize=(17, 3))
        # ax.scatter(events[:, 0] / raw.info['sfreq'], events[:, 2], marker=".")
        # ax.scatter(onsets[:, 0] / raw.info['sfreq'], onsets[:, 2], marker="x")

        # Slice data to trials
        epochs.append(mne.Epochs(raw, events=onsets, tmin=tmin, tmax=tmax, baseline=None, picks="eeg"))

        # Read labels
        # N.B. Minus 1 to convert Matlab counting from 1 to Python counting from 0
        f = h5py.File(os.path.join(path, "sourcedata", "offline", subject, block, "trainlabels.mat"), "r")
        labels.append(np.array(f["v"]).astype("uint8").flatten() - 1)

    # Extract data and concatenate runs
    X = mne.concatenate_epochs(epochs).get_data().astype("float32")[:, :, :int((tmax-tmin)*fs)]
    y = np.array(labels).flatten().astype("uint8")

    # Create output folder
    if not os.path.exists(os.path.join(path, "derivatives300", "offline", subject)):
        os.makedirs(os.path.join(path, "derivatives300", "offline", subject))

    # Save data
    np.savez(os.path.join(path, "derivatives300", "offline", subject, f"{subject}_gdf.npz"), 
            X=X, y=y, V=V, fs=fs)

    # Print summary
    print("Subject: ", subject)
    print("\tX shape:", X.shape, "(trials, channels, samples)")
    print("\ty shape:", y.shape, "(trials)")
    print("\tV shape:", V.shape, "(samples, codes)")
    print("\tfs: ", fs)

Extracting EDF parameters from D:\Dataset\sourcedata\offline\sub-01\block_1\sub-01_20181128_block_1_main_eeg.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
AF3, F3, FC5, P7, P8, FC6, F4, AF4
Creating raw.info structure...
Reading 0 ... 341478  =      0.000 ...   666.949 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 845 samples (1.650 s)

320 events found
Event IDs: [257]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished


320 events found
Event IDs: [257]
320 events found
Event IDs: [257]
Not setting metadata
20 matching events found
No baseline correction applied
0 projection items activated
Extracting EDF parameters from D:\Dataset\sourcedata\offline\sub-01\block_2\sub-01_20181128_block_2_main_eeg.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
AF3, F3, FC5, P7, P8, FC6, F4, AF4
Creating raw.info structure...
Reading 0 ... 341106  =      0.000 ...   666.223 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 30.00 Hz
- U

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.0s finished


320 events found
Event IDs: [257]
320 events found
Event IDs: [257]
Not setting metadata
20 matching events found
No baseline correction applied
0 projection items activated
Extracting EDF parameters from D:\Dataset\sourcedata\offline\sub-01\block_3\sub-01_20181128_block_3_main_eeg.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
AF3, F3, FC5, P7, P8, FC6, F4, AF4
Creating raw.info structure...
Reading 0 ... 341104  =      0.000 ...   666.219 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 30.00 Hz
- U

...

320 events found
Event IDs: [257]
320 events found
Event IDs: [257]
Not setting metadata
20 matching events found
No baseline correction applied
0 projection items activated
Extracting EDF parameters from D:\Dataset\sourcedata\offline\sub-30\block_5\sub-30_20190311_block_5_main_eeg.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
AF3, F3, FC5, P7, P8, FC6, F4, AF4
Creating raw.info structure...
Reading 0 ... 341073  =      0.000 ...   666.158 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 30.00 Hz
- U

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


320 events found
Event IDs: [257]


[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.2s finished


320 events found
Event IDs: [257]
320 events found
Event IDs: [257]
Not setting metadata
20 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 20 events and 9451 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 20 events and 9451 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 20 events and 9451 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 20 events and 9451 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 20 events and 9451 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 20 events and 9451 original time points ...
Using data from preloaded Raw for 20 events and 9451 original time points ...
Using data from preloaded Raw for 20 events and 9451 original time points ...
Using data from preloaded Raw for 20 events and 9451 original time points ...
Using data from pre